In [ ]:
#| default_exp src.raw

# Raw Data Conversion
> raw file conversion & analysis (created on April 22nd 2022 by M. Reichmann (remichae@phys.ethz.ch))

In [ ]:
#| export
#!/usr/bin/env python

In [ ]:
#| export

from subprocess import check_call
import h5py
from numpy import all, array
from pathlib import Path

from fastcore.script import *

from HighResAnalysis.plotting.utils import prep_kw, choose, remove_file, info
from HighResAnalysis.plotting.draw import Draw
from HighResAnalysis.plotting.fit import Gauss
from HighResAnalysis.src.converter import Converter, Raw
from HighResAnalysis.utility.affine_transformations import transform
from HighResAnalysis.src.dut import Plane
from HighResAnalysis.src.analysis import Analysis

In [ ]:
#| export
@call_parse
def main():
    c_ = Converter.from_ana(41, 1)
    z = Raw(c_, load_file=True, step=2)

In [ ]:
#| hide
from nbdev import *
nbdev_export()